# Install Required Packages

In [2]:
!pip install pyathena
!pip3 install -U sagemaker


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# Import Required Libraries

In [3]:
import boto3 # aws sdk for python
import sagemaker # machine learning platform
import numpy as np # array manipulation
import os # operating system interfaces
import pandas as pd # python data analysis
import re # regular expressions
from pyathena import connect # athena client
from sagemaker.pytorch.estimator import PyTorch # PyTorch estimator
from time import gmtime, strftime, sleep # time-related functions

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


# Perform Prerequisites

In [4]:
# establish sagemaker session, provide permissions
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# create a boto3 session for the sagemaker service
sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# client to make featurestore record calls
featurestore_runtime = boto3.Session().client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

# create boto3 session to establish feature store session
boto_session = boto3.Session(region_name=region)

# create featurestore session
feature_store_session = sagemaker.Session(
    boto_session=boto_session,
    sagemaker_client=sm,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

# define prefixes for the safety data directory and featurestore
prefix_data = 'safety/data'
prefix_featurestore = 'safety-featurestore'

# print s3 locations
print('Data directory location:', f"s3://{bucket}/{prefix_data}")
print('FeatureStore directory location:', f"s3://{bucket}/{prefix_featurestore}\n")

# print current IAM role for notebook instance
role = sagemaker.get_execution_role()
print('Execution Role:', role)

Data directory location: s3://sagemaker-us-east-1-752648173624/safety/data
FeatureStore directory location: s3://sagemaker-us-east-1-752648173624/safety-featurestore

Execution Role: arn:aws:iam::752648173624:role/LabRole


# Data Preparation

## Query Catalog Data

In [5]:
# define database name
database_name = 'safetydb'

# define table name
table_name_csv = 'catalog_csv'

# set s3 temporary staging directory
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

# define connection parameters
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# define sql query statement
statement = """SELECT * FROM {}.{}
    WHERE img_filename like '%.jpg'
    AND label_filename like '%.txt'
    LIMIT 25000""".format(
    database_name, table_name_csv
)

# print sql statement for review before executing
print('SQL query SELECT statement:\n', statement)

SQL query SELECT statement:
 SELECT * FROM safetydb.catalog_csv
    WHERE img_filename like '%.jpg'
    AND label_filename like '%.txt'
    LIMIT 25000


In [6]:
# execute sql query and display results
df_catalog_query = pd.read_sql(statement, conn)
df_catalog_query.head(10)

/tmp/ipykernel_5724/3145676702.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_catalog_query = pd.read_sql(statement, conn)


,sample_id,img_filename,label_filename
0,000001,000001.jpg,000001.txt
1,000002,000002.jpg,000002.txt
2,000003,000003.jpg,000003.txt
3,000004,000004.jpg,000004.txt
4,000005,000005.jpg,000005.txt
5,000006,000006.jpg,000006.txt
6,000007,000007.jpg,000007.txt
7,000008,000008.jpg,000008.txt
8,000009,000009.jpg,000009.txt
9,000010,000010.jpg,000010.txt


## Combine with FeatureGroup Data

### Get Batch Records from FeatureGroups

In [7]:
# define featuregroup name patterns
image_feature_group_name_pattern = 'image-feature-group-'
label_feature_group_name_pattern = 'label-feature-group-'

# obtain latest version of featuregroup names, if multiples exist
all_image_feature_groups = sm.list_feature_groups(NameContains=image_feature_group_name_pattern, SortBy='CreationTime', SortOrder='Descending')
all_label_feature_groups = sm.list_feature_groups(NameContains=label_feature_group_name_pattern, SortBy='CreationTime', SortOrder='Descending')
image_feature_group_name = all_image_feature_groups['FeatureGroupSummaries'][0]['FeatureGroupName']
label_feature_group_name = all_label_feature_groups['FeatureGroupSummaries'][0]['FeatureGroupName']

# print featuregroup names
print('Image FeatureGroup Name:', image_feature_group_name)
print('Label FeatureGroup Name:', label_feature_group_name)

Image FeatureGroup Name: image-feature-group-16-21-22-08
Label FeatureGroup Name: label-feature-group-16-21-22-08


In [8]:
# specify record identifiers (i.e., sample ids) from previous catalog query
record_identifiers_value = df_catalog_query['sample_id'].values.astype(str).tolist()

# query image featuregroup by using record_id as primary key
df_image_feature_group = featurestore_runtime.batch_get_record(
    Identifiers=[
        {
            'FeatureGroupName': image_feature_group_name,
            'RecordIdentifiersValueAsString':record_identifiers_value,
        }
    ]
)

# query label featuregroup by using record_id as primary key
df_label_feature_group = featurestore_runtime.batch_get_record(
    Identifiers=[
        {
            'FeatureGroupName': label_feature_group_name,
            'RecordIdentifiersValueAsString': record_identifiers_value,
        }
    ]
)

ClientError: An error occurred (ValidationException) when calling the BatchGetRecord operation: 1 validation error detected: Value '[000001, 000002, 000003, 000004, 000005, 000006, 000007, 000008, 000009, 000010, 000011, 000012, 000013, 000014, 000015, 000016, 000017, 000018, 000019, 000020, 000021, 000022, 000023, 000024, 000025, 000026, 000027, 000028, 000029, 000030, 000031, 000032, 000033, 000034, 000035, 000036, 000037, 000038, 000039, 000040, 000041, 000042, 000043, 000044, 000045, 000046, 000047, 000048, 000049, 000050, 000051, 000052, 000053, 000054, 000055, 000056, 000058, 000059, 000060, 000061, 000062, 000064, 000065, 000066, 000067, 000068, 000069, 000070, 000072, 000073, 000074, 000076, 000077, 000078, 000079, 000080, 000081, 000082, 000083, 000084, 000085, 000086, 000087, 000089, 000090, 000091, 000092, 000093, 000094, 000096, 000097, 000098, 000099, 000100, 000101, 000102, 000103, 000104, 000106, 000107, 000108, 000109, 000110, 000111, 000112, 000113, 000114, 000115, 000

### Display Image FeatureGroup Records

In [ ]:
# create list of image records and feature names, exclude eventtime
image_records = [sample['Record'] for sample in df_image_feature_group['Records']]
image_feature_names = [feature['FeatureName'] for feature in image_records[0] if feature['FeatureName'] != 'EventTime']

# create list of image data
image_data = [image_feature_names]

# iterate through each record in image featuregroup
for record in image_records:
    
    # iterate through each feature in individual record
    image_data.append([feature['ValueAsString'] for feature in record if feature['FeatureName'] != 'EventTime'])
                       
# create images dataframe
df_images = pd.DataFrame(image_data[1:], columns=image_data[0])
                       
# display dataframe head
df_images.head()

### Display Label FeatureGroup Records

In [ ]:
# create list of label records and feature names, exclude eventtime
label_records = [sample['Record'] for sample in df_label_feature_group['Records']]
label_feature_names = [feature['FeatureName'] for feature in label_records[0] if feature['FeatureName'] != 'EventTime']

# create list of label data
label_data = [label_feature_names]

# iterate through each record in label featuregroup
for record in label_records:
    
    # iterate through each feature in individual record
    label_data.append([feature['ValueAsString'] for feature in record if feature['FeatureName'] != 'EventTime'])
                       
# create labels dataframe
df_labels = pd.DataFrame(label_data[1:], columns=label_data[0])
                       
# display dataframe head
df_labels.head()

### Join Records

In [ ]:
# merge catalog query and images dataframes, then the resulting with labels dataframe
df_combined = pd.merge(df_catalog_query, df_images, on='sample_id')
df_combined = pd.merge(df_combined, df_labels, on='sample_id')

# display resulting datafrmae
df_combined.head()

## Split Data

In [ ]:
# use a copy of dataframe, can manipulate if desired for experimentation
data = df_combined.copy()

# data split in four sets - training, validation, test, and batch inference
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.4
val_list = (rand_split >= 0.4) & (rand_split < 0.5)
test_list = (rand_split >= 0.5) & (rand_split < 0.6)
batch_list = rand_split >= 0.6 # "production" data

# print data splits
print('Data Splits:')
print('------------')
print(f"Train :   {sum(train_list)} samples")
print(f"Val   :   {sum(val_list)} samples")
print(f"Test  :   {sum(test_list)} samples")
print(f"Batch :   {sum(batch_list)} samples")

## Create Split Datasets in S3

In [ ]:
# define and print source s3 locations
s3_images_source = f"s3://{bucket}/{prefix_data}/images/"
s3_labels_source = f"s3://{bucket}/{prefix_data}/labels/"
print('Images source directory location:', s3_images_source)
print('Labels source directory location:', s3_labels_source, '\n')

# define and print destination s3 location for data splits
s3_split_dest = f"s3://{bucket}/{prefix_data}/split/"
print('Split destination directory location:', s3_split_dest)

In [ ]:
# define function to copy files for respective data splits to corresponding s3 destinations
# provide 'split_name' as either 'train', 'val', 'test', or 'batch'
# provide 'split_list' as either 'train_list', 'val_list', 'test_list', or 'batch_list'
def split_dataset(split_name, split_list):

    # iterate through each sample in split
    for index, sample in data[split_list].iterrows():

        # source/destination variables for individual sample
        cp_image_source = f"{s3_images_source}{sample['img_filename']}"
        cp_image_dest = f"{s3_split_dest}{split_name}/images/"
        cp_label_source = f"{s3_labels_source}{sample['label_filename']}"
        cp_label_dest = f"{s3_split_dest}{split_name}/labels/"

        # copy from source to destination
        !aws s3 cp $cp_image_source $cp_image_dest
        !aws s3 cp $cp_label_source $cp_label_dest

In [ ]:
# perform data copies
print('Beginning TRAIN data split copies.')
split_dataset(split_name='train', split_list=train_list)
print('Completed TRAIN data split copies.\n')

print('Beginning VAL data split copies.')
split_dataset(split_name='val', split_list=val_list)
print('Completed VAL data split copies.\n')

print('Beginning TEST data split copies.')
split_dataset(split_name='test', split_list=test_list)
print('Completed TEST data split copies.\n')

print('Beginning BATCH data split copies.')
split_dataset(split_name='batch', split_list=batch_list)
print('Completed BATCH data split copies.')

# Training Job and Model Creation

## Create and Run Training Job

In [ ]:
%%time

# define job name and output location
job_name = 'yolov8-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(bucket, prefix_data, job_name)

# build a PyTorch estimator
pytorch_estimator = PyTorch(
    role=role,
    entry_point='train.py', # custom training script, locate in code directory
    framework_version='2.0.1', # training - CPU - Python 3.10
    py_version='py310',
    source_dir='./code',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=output_location,
    sagemaker_session=sess,
    hyperparameters = {'data': 'data.yaml', # yaml config file for custom dataset
                       'epochs': 5, # number of training epochs
                       'yolo_model': 'yolov8n.pt', # pretrained base model
                       'saved_model_name': 'model.pt' # name for model export
                      }
)

# s3 location where training data is saved
inputs = s3_split_dest[:-1]

# begin training job
pytorch_estimator.fit(inputs=inputs, job_name=job_name, logs='All')

## Batch Transform

In [ ]:
# create a transform job...

In [ ]:
%%time
from sagemaker.pytorch.model import PyTorchModel # PyTorch model

# define paths
model_data = f"{output_location}/{job_name}/output/model.tar.gz" # trained model artifacts
transformer_input = f"{s3_split_dest}batch/images" # batch directory for input data
transformer_output = f"{output_location}/{job_name}/output/transformer" # transformer job results

# create a PyTorch model
pytorch_model = PyTorchModel(
    model_data=model_data, # trained model artifacts
    role=role,
    entry_point='inference.py', # custom inference script, locate in code directory
    framework_version='2.1.0', # CPU - Python 3.10
    py_version='py310',
    source_dir='code',
    sagemaker_session=sess
)

# create a transformer from the PyTorch model
transformer = pytorch_model.transformer(
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=transformer_output,
    accept='application/json',
    max_payload=10
)

# begin batch transform job
transformer.transform(
    data=transformer_input,
    content_type='image/jpeg'
)

# wait for job to complete
transformer.wait()

In [ ]:
import json # encoder and decoder
# define s3 client
s3_client = boto3.client("s3", region_name=region)

# define prefix for transformer job output
transformer_output_s3_prefix = transformer_output.replace(
    f"s3://{bucket}/", ""
)

# get job output files from bucket
objects_in_bucket = s3_client.list_objects(
    Bucket=bucket, Prefix=transformer_output_s3_prefix
)

# define function to convert an s3 file to dataframe
def s3_file_to_df(key):
    
    # get single s3 file
    s3_file = s3_client.get_object(Bucket=bucket, Key=key)
    
    # decode, read, and create list of json strings
    body = s3_file['Body'].read().decode('utf-8')
    json_strings = json.loads(body)
    
    # create list for data to create dataframe
    data_list = []

    # parse json strings and append to data list
    for json_str in json_strings:
        json_obj = json.loads(json_str)
        data_list.append(json_obj)
        
    # create dataframe
    df = pd.DataFrame(data_list)
    
    # insert column for sample id, obtained from s3 filename
    df.insert(0, 'sample_id', key.split("/")[-1].split(".")[0])
    return df

# create list to store dataframes for concatenation
df_list = []

# iterate through each object in s3 bucket
for obj in objects_in_bucket.get('Contents', []):
    
    # get filename
    s3_filename = obj['Key']

    # skip over any files without expected '.out' file extension
    if s3_filename.endswith('.out'):

        # call function to create dataframe of current file contents
        df_current_file = s3_file_to_df(s3_filename)
        
        # insert column for timestamp
        df_current_file.insert(0, 'timestamp', obj['LastModified'])

        # append to dataframe list for concatenation
        df_list.append(df_current_file)

# concatenate dataframes to create end result
df_batch_transform = pd.concat(df_list, ignore_index=True)

# convert class id to int
df_batch_transform['class_id'] = df_batch_transform['class_id'].astype(int)

# display dataframe
df_batch_transform

# Example Business Problem Query

Query the dataframe to obtain number of detections of each class we are interested in.

In [ ]:
# create new dataframe copy
df_business_query = df_batch_transform.copy()

# add a date column in YYYY-MM-DD format derived from timestamp
df_business_query['date'] = df_batch_transform['timestamp'].dt.date

# create new dataframe with business query results
df_business_query = pd.DataFrame(df_business_query.groupby(['date', 'class_id', 'class_name']).size().reset_index(name='detections'))

# filter out class id of 1 for 'vest'
df_business_query = df_business_query.query('class_id != 1')

# display dataframe
df_business_query

# Monitors
Quality monitors help detect issues such as concept drift, data drift, or model degradation over time, enabling timely corrective actions to maintain the performance and reliability of machine learning systems.

https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html

In [ ]:
# imports
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker import get_execution_role, session, Session
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import CronExpressionGenerator
from sagemaker.transformer import Transformer
from sagemaker.inputs import BatchDataCaptureConfig

### Capture Data

In [ ]:
# Define the S3 URI where captured data will be stored
data_capture_destination = "s3://captured_data_S3_uri" # Replace with actual S3 URI

# Define the model name for data capture
model_name = "monitor_model" # Replace with actual model name

# Initialize a Transformer object for capturing data during inference
transformer = Transformer(model_name=model_name)
transform_arg = transformer.transform(
    batch_data_capture_config=BatchDataCaptureConfig(
        destination_s3_uri=data_capture_destination,
        kms_key_id="kms_key", # Replace with actual KMS key ID
        generate_inference_id=True,
    ),
)

In [ ]:
# Define paths for captured data, output reports, ground truth, statistics, and constraints
s3_capture_upload_path = "s3://captured_data_upload_path"  # Replace with actual S3 paths
s3_report_path = "s3://report_output_path"
gt_s3_uri = "s3://ground_truth_data_path"

statistics_path = "statistics.json"
constraints_path = "constraints.json"

Once the transform job completes, the captured data gets logged under the DestinationS3Uri provided with the data capture configuration.

---------

### Implement data Quality monitors on ML system


#### Create Baseline
Create a baseline job that compares your model predictions with ground truth labels in a baseline dataset that you have stored in Amazon S3.

In [ ]:
baseline_results_uri = # location of the baseline results
baseline_data = # locaion of the baseline data

In [ ]:
# Define and create a DefaultModelMonitor for data quality monitoring
data_quality_model_monitor = DefaultModelMonitor(role=get_execution_role())

# Create a baseline job to suggest baseline statistics and constraints
baseline_results_uri = "s3://baseline_results_path"  # Replace with actual S3 URI
baseline_data = "s3://baseline_data_path"  # Replace with actual S3 URI

data_quality_model_monitor.suggest_baseline(
    baseline_dataset=baseline_data,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)

#### Schedule data quality monitoring jobs

In [ ]:
# Schedule data quality monitoring jobs
baseline_data_job_name = "baseline_data_job"  # Define the baseline job name

schedule = data_quality_model_monitor.create_monitoring_schedule(
    monitor_schedule_name=baseline_data_job_name,
    batch_transform_input=BatchTransformInput(
        data_captured_destination_s3_uri=s3_capture_upload_path,
        destination="/opt/ml/processing/input", # ~not sure
        dataset_format=MonitoringDatasetFormat.csv(header=False),
    ),
    output_s3_uri=s3_report_path,
    statistics=statistics_path,
    constraints=constraints_path,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

-------------------

### Implement model Quality monitors on ML system.

#### Create Baseline
Create a baseline job that compares your model predictions with ground truth labels in a baseline dataset that you have stored in Amazon S3.

In [ ]:
# Define and create a ModelQualityMonitor for model quality monitoring
model_quality_monitor = ModelQualityMonitor(
    role=get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=Session()
)

In [ ]:
# Create a baseline job to suggest baseline statistics and constraints for model quality monitoring
baseline_results_uri = "s3://baseline_results_path"  # Replace with actual S3 URI
baseline_data = "s3://baseline_data_path"  # Replace with actual S3 URI
baseline_model_job_name = "baseline_model_job"  # Define the baseline job name

job = model_quality_monitor.suggest_baseline(
    job_name=baseline_model_job_name,
    baseline_dataset=baseline_data,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type='BinaryClassification',
    inference_attribute="class_name", # The column in the dataset that contains predictions. ~not sure
    probability_attribute="confidence", # The column in the dataset that contains probabilities. ~not sure
    ground_truth_attribute="class_id" # The column in the dataset that contains ground truth labels. ~not sure
)
job.wait(logs=False)

#### Schedule data quality monitoring jobs

In [ ]:
model_quality_model_monitor = ModelQualityMonitor(
   role=sagemaker.get_execution_role(),
)

# Schedule model quality monitoring jobs
mon_schedule_name = "model_quality_schedule"  # Define the model quality monitoring schedule name
schedule = model_quality_model_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    batch_transform_input=BatchTransformInput(
        data_captured_destination_s3_uri=s3_capture_upload_path,
        destination="/opt/ml/processing/input", #change (storage location?)
        dataset_format=MonitoringDatasetFormat.csv(header=False),
        probability_attribute="0",
        probability_threshold_attribute=0.5, # the threshold to classify the inference probablity to class 0 or 1 in 
        start_time_offset="-PT6H", # look back 6 hour for transform job outputs.
        end_time_offset="-PT0H"
    ),
    ground_truth_input=gt_s3_uri,
    output_s3_uri=s3_report_path,
    problem_type="BinaryClassification",
    constraints=constraints_path, #baseline_job.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

-----------

### Implement infrastructure monitors on ML system

https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Lambda-Insights-Getting-Started.html

### Create a monitoring dashboard for your ML endpoint/job on CloudWatch

https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/create_dashboard.html

### Generate model and data reports on SageMaker.

from the dashboard